<a href="https://colab.research.google.com/github/MarynaZinchenko888/python_for_ds_tasks/blob/main/Zinchenko_M_%22HW_%D0%9F%D0%B5%D1%80%D0%B5%D0%B2%D1%96%D1%80%D0%BA%D0%B0_%D1%81%D1%82%D0%B0%D1%82%D0%B8c%D1%82%D0%B8%D1%87%D0%BD%D0%B8%D1%85_%D0%B3%D1%96%D0%BF%D0%BE%D1%82%D0%B5%D0%B7_Z_test_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Перевірка статиcтичних гіпотез. Z-test.



0. Зчитайте дані з `data.csv` у змінну data, яка має тип pandas.DataFrame. Ми будемо далі працювати з цією змінною.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Налаштування для візуалізації
plt.rcParams['figure.figsize'] = (12, 7)
plt.rcParams['font.size'] = 12


df = pd.read_csv("data.csv")
df

,Unnamed: 0,data
0,0,17.499453
1,1,19.662399
2,2,7.182823
3,3,29.841625
4,4,9.239386
...,...,...
95,95,26.244944
96,96,22.282832
97,97,18.697188
98,98,27.041189


Запустіть код нижче. Для коректної робити всіх подальших методів дані мають бути у вигляді одновимірного `numpy.ndarray` вектора та мати розмірність (100,). Така розмірність означає, що дані - одновимірні (колонка), якщо б розмірність була (100, 1), то дані сприймаються методами як двовимірні (таблиця), хоча для нас виглядати можуть так само.

In [4]:
data = df["data"].values
data = data.ravel()

In [5]:
print(type(data))
data.shape

<class 'numpy.ndarray'>


(100,)

**Завдання 1**. Зчитані дані - це сума покупок в доларах наших покупців на сайті протягом липня 2025 року.

До цього місяця, ми мали продажі в середньому на чек $20.

Необхідно зʼясувати, чи вийшло у нас статистично значущо **підвищити** середній чек за липень 2025?

Аби дати відповідь - ми проведемо z-test на рівні значущості $\alpha=0.05$ двома способами. В цьому завданні ми будемо виконувати обчислення "вручну" з використанням бібліотек numpy та scipy.stats подібно до прикладу в лекції. Для цього виконайте наступні 7 кроків. Правильне виконання кожного з кроків оцінюється в 1 бал.

1. Запишіть параметри задачі у змінні Python:
    - `sample_mean` - $\bar{x}$, середнє значення в вибірці
    - `population_mean` - $\mu_0$, середнє значення в популяції (тобто те, з яким ми порівнюємо середнє вибіркове значення)
    - `population_std` - $\sigma$, вибіркове стандартне відхилення, яке ми вважаємо, що є рівним ст. відх. популяції, адже маємо достатньо велику вибірку.
    - `sample_size` - $n$, розмір вибірки
    - `alpha` - $\alpha$ рівень значущості
    
    **Увага!** Для обчислення стандартного відхилення маємо скористатись функцією `np.std(your_dataframe, ddof=1)`. Чому так - розбираємо в лекції про t-test.


2. Визначте для цієї задачі:
    - якою є гіпотеза $H_0$
    - якою є альтернативна гіпотеза $H_a$
    - з яким типом тесту ми маємо справу - лівосторонній, правосторонній чи двосторонній.

3. Обчисліть стандартну помилку SE.
4. Розрахуйте z-статистику.
5. Знайдіть критичне z-значення з допомогою бібліотеки stats.
6. Обчисліть p-value з допомогою бібліотеки stats.
7. Прийміть рішення, чи відхиляєте ви гіпотезу $H_0$. Для прийняття рішення зробіть і порівняння z-статистики з критичним значенням, і проаналізуйте p-value.

### Завдання 1.1: Визначаємо параметри задачі



In [12]:
sample_mean = np.mean(df["data"].values)                # x̄ - середнє значення у вибірці
population_mean = 20                                    # μ₀ - середнє значення в популяції (тобто те, з яким ми порівнюємо середнє вибіркове значення)
population_std = np.std(df.values.ravel(), ddof=1)      # σ - стандартне відхилення в популяції
sample_size = len(data)                                 # n , розмір вибірки
alpha = 0.05                                            # α - рівень значущості (ймовірність помилки I роду)

In [16]:
print("Середнє значення у вибірці: x̄ =", sample_mean)
print("Середнє значення в популяції: μ0 =", population_mean)
print("Стандартне відхилення в популяції: σ =", population_std)
print("Розмір вибірки: n =", sample_size)
print("Рівень значущості: α =", alpha)

Середнє значення у вибірці: x̄ = 19.37755319670441
Середнє значення в популяції: μ0 = 20
Стандартне відхилення в популяції: σ = 25.809963197550587
Розмір вибірки: n = 100
Рівень значущості: α = 0.05



### Завдання 1.2: Формулюємо гіпотези

**Гіпотези**   

H₀: μ = 20 (середній чек не змінився)  
H₁: μ > 20 (середній чек збільшився)  
Тип тесту: правосторонній (one-tailed)  

**Чому саме правосторонній тест?**

Тому, що потрібно визначити, чи вийшло у нас статистично значущо **підвищити** середній чек за липень 2025.


### Завдання 1.3: Обчислюємо стандартну помилку


- σ = 25.80996 - стандартне відхилення IQ в популяції (наскільки розкидані IQ окремих людей)
- SE = σ/√n = 25.80996/√100 ≈ 2.5810 - стандартна помилка (наскільки в середньому відхиляється середній чек вибірки зі 100 покупок від справжнього середнього)

In [17]:
# Формула: SE = σ / √n
standard_error = population_std / np.sqrt(sample_size)

print("=" * 60)
print("ОБЧИСЛЕННЯ СТАНДАРТНОЇ ПОМИЛКИ:")
print("=" * 60)
print("Формула: SE = σ / √n")
print(f"SE = {population_std} / √{sample_size}")
print(f"SE = {population_std} / {np.sqrt(sample_size):.3f}")
print(f"SE = {standard_error:.3f}")
print()
print("Інтерпретація: В середньому, вибіркове середнє відхиляється")
print(f"від справжнього середнього на ±{standard_error:.3f} доларів США")

ОБЧИСЛЕННЯ СТАНДАРТНОЇ ПОМИЛКИ:
Формула: SE = σ / √n
SE = 25.809963197550587 / √100
SE = 25.809963197550587 / 10.000
SE = 2.581

Інтерпретація: В середньому, вибіркове середнє відхиляється
від справжнього середнього на ±2.581 доларів США


### Завдання 1.4: Розрахунок z-статистики

In [18]:
# Формула: z = (x̄ - μ₀) / SE
z_statistic = (sample_mean - population_mean) / standard_error

print("=" * 60)
print("РОЗРАХУНОК Z-СТАТИСТИКИ:")
print("=" * 60)
print("Формула: z = (x̄ - μ₀) / SE")
print(f"z = ({sample_mean} - {population_mean}) / {standard_error:.3f}")
print(f"z = {sample_mean - population_mean} / {standard_error:.3f}")
print(f"z = {z_statistic:.3f}")
print()
print("Інтерпретація:")
print(f"Наше вибіркове середнє відхиляється на {z_statistic:.3f} стандартних")
print("помилок від гіпотетичного середнього популяції.")
print()
if z_statistic > 2:
    print("Це досить велике відхилення. (більше 2 стандартних помилок)")
elif z_statistic > 1:
    print("Це помірне відхилення (між 1 та 2 стандартними помилками)")
else:
    print("Це невелике відхилення (менше 1 стандартної помилки)")

РОЗРАХУНОК Z-СТАТИСТИКИ:
Формула: z = (x̄ - μ₀) / SE
z = (19.37755319670441 - 20) / 2.581
z = -0.6224468032955883 / 2.581
z = -0.241

Інтерпретація:
Наше вибіркове середнє відхиляється на -0.241 стандартних
помилок від гіпотетичного середнього популяції.

Це невелике відхилення (менше 1 стандартної помилки)


### Завдання 1.5: Знайти критичне z-значення за допомогою бібліотеки stats.

In [19]:
# Для правостороннього тесту з α = 0.05
z_critical = stats.norm.ppf(1 - alpha)

print("=" * 60)
print("КРИТИЧНЕ ЗНАЧЕННЯ:")
print("=" * 60)
print(f"Для рівня значущості α = {alpha} (правосторонній тест)")
print(f"Критичне z-значення = {z_critical:.3f}")
print()
print("Що це означає?")
print(f"Якщо z-статистика > {z_critical:.3f}, то результат настільки")
print("незвичайний, що ми не віримо у випадковість за гіпотезою Н0.")
print(f"Ймовірність отримати z > {z_critical:.3f} випадково = {alpha*100}%")

КРИТИЧНЕ ЗНАЧЕННЯ:
Для рівня значущості α = 0.05 (правосторонній тест)
Критичне z-значення = 1.645

Що це означає?
Якщо z-статистика > 1.645, то результат настільки
незвичайний, що ми не віримо у випадковість за гіпотезою Н0.
Ймовірність отримати z > 1.645 випадково = 5.0%


### Завдання 1.6: Обчислити p-value за допомогою бібліотеки stats.

In [20]:
# Для правостороннього тесту: p-value = P(Z > z_statistic)
p_value = 1 - stats.norm.cdf(z_statistic)

print("=" * 60)
print("P-VALUE:")
print("=" * 60)
print(f"p-value = {p_value:.4f}")
print()
print("Що це означає?")
print(f"Якщо H₀ правильна (середній чек не змінився), то ймовірність")
print(f"випадково отримати середній чек ≥ {sample_mean} становить {p_value:.4f}")
print(f"або приблизно {p_value*100:.4f}%")
print()
if p_value < 0.001:
    print("Це НАДЗВИЧАЙНО малоймовірно! (менше 0.1%)")
elif p_value < 0.01:
    print("Це дуже малоймовірно! (менше 1%)")
elif p_value < 0.05:
    print("Це малоймовірно! (менше 5%)")
else:
    print("Це цілком можливо випадково")

P-VALUE:
p-value = 0.5953

Що це означає?
Якщо H₀ правильна (середній чек не змінився), то ймовірність
випадково отримати середній чек ≥ 19.37755319670441 становить 0.5953
або приблизно 59.5287%

Це цілком можливо випадково


### 1.7 Прийняття рішення щодо відхилення гіпотези $H_0$.


Метод 1: Порівняння z-статистики з критичним значенням

In [21]:
print("=" * 60)
print("МЕТОД 1: Порівняння z-статистики з критичним значенням")
print("=" * 60)
print(f"z-статистика = {z_statistic:.3f}")
print(f"Критичне значення = {z_critical:.3f}")
print(f"Порівняння: {z_statistic:.3f} {'>' if z_statistic > z_critical else '≤'} {z_critical:.3f}")

if z_statistic > z_critical:
    print("Висновок: z-статистика ПЕРЕВИЩУЄ критичне значення")
    decision1 = "ВІДХИЛЯЄМО H₀"
else:
    print("Висновок: z-статистика НЕ перевищує критичне значення")
    decision1 = "НЕ ВІДХИЛЯЄМО H₀"

print(f"Рішення: {decision1}")

МЕТОД 1: Порівняння z-статистики з критичним значенням
z-статистика = -0.241
Критичне значення = 1.645
Порівняння: -0.241 ≤ 1.645
Висновок: z-статистика НЕ перевищує критичне значення
Рішення: НЕ ВІДХИЛЯЄМО H₀


Метод 2: Порівняння p-value з рівнем значущості



In [25]:
print("\n" + "=" * 60)
print("МЕТОД 2: Порівняння p-value з рівнем значущості")
print("=" * 60)
print(f"p-value = {p_value:.6f}")
print(f"Рівень значущості α = {alpha}")
print(f"Порівняння: {p_value:.6f} {'<' if p_value < alpha else '≥'} {alpha}")

if p_value < alpha:
    print("Висновок: p-value МЕНШЕ за рівень значущості")
    decision2 = "ВІДХИЛЯЄМО H₀"
else:
    print("Висновок: p-value НЕ менше за рівень значущості")
    decision2 = "НЕ ВІДХИЛЯЄМО H₀"

print(f"Рішення: {decision2}")


МЕТОД 2: Порівняння p-value з рівнем значущості
p-value = 0.595287
Рівень значущості α = 0.05
Порівняння: 0.595287 ≥ 0.05
Висновок: p-value НЕ менше за рівень значущості
Рішення: НЕ ВІДХИЛЯЄМО H₀


***Отже, середній чек за липень 2025 року не підвищився статистично значущо.
Невелике зниження до ≈19.38 від 20 цілком пояснюється випадковими коливаннями у вибірці.***

**Завдання 2.** Виконайте обчислення z-test з використанням бібліотеки statsmodels.

Отримайте z-статистику та р-значення.

Виведіть p-значення та зробіть висновок, чи ми маємо достатньо доказів, аби стверджувати, що середній чек зріс.

Чи зійшлись значення z-статистику та р-значення в цьому завданні з попередніми обчисленнями?

Завдання оцінюється в 3 бали.

In [37]:
from statsmodels.stats import weightstats as stests

print("=" * 60)
print("ВИКОРИСТАННЯ STATSMODELS ДЛЯ Z-ТЕСТУ:")
print("=" * 60)

np.random.seed(95)  # Для відтворюваності результатів
sample_data = np.random.normal(sample_mean, population_std, sample_size)

# Коригуємо, щоб середнє було точно 110 (як у задачі)
sample_data = sample_data - np.mean(sample_data) + sample_mean

ВИКОРИСТАННЯ STATSMODELS ДЛЯ Z-ТЕСТУ:


In [38]:
sample_data

array([  3.46074887,   5.57350273,  29.37055493,  36.07364053,
        48.70120099,  23.11419184,  16.93240655,  66.59286527,
        35.78538495,  20.88303218, 109.79535357,  -9.19750081,
        25.53892125,  18.9685016 ,   5.32640963, -17.92467788,
        10.4378014 ,  49.86328673,  25.39808167, -19.60676157,
        12.80567296,  17.37617943,  45.46114821,   4.28491649,
        -1.14645653,   2.67565429,  26.88148838,  19.831103  ,
        38.97751148,  13.79826004, -16.14588742, -12.99562052,
        16.59780851,  20.23019342,   3.01124911,  24.03073067,
         9.42835363,  -0.70807995,  15.09132471,  31.08853194,
        16.97350029, -20.59104138,  46.16151865,  30.46768175,
        52.77683517,   8.81918219,   3.25839106,  12.36413927,
         6.95494067,  23.4442503 ,   7.07488584,  23.91831578,
        33.56375506,  20.47491015,  19.29376589,  35.33498514,
       -29.31836172,  38.31398869,  25.56022045, -18.8803832 ,
        49.53829722,  35.71340023,  31.26510751, -10.38

In [39]:
np.mean(sample_data)

np.float64(19.37755319670441)

In [40]:
print(f"Згенеровані дані (симуляція 100 чеків):")
print(f"  Кількість чеків (розмір вибірки): {len(sample_data)}")
print(f"  Середній чек: {np.mean(sample_data):.2f}")
print(f"  Стандартне відхилення: {np.std(sample_data, ddof=1):.2f}")

Згенеровані дані (симуляція 100 чеків):
  Кількість чеків (розмір вибірки): 100
  Середній чек: 19.38
  Стандартне відхилення: 23.54


In [41]:
print("\n" + "=" * 60)
print("ВИКОНАННЯ Z-ТЕСТУ:")
print("=" * 60)

# Виконуємо правосторонній z-тест
z_stat_sm, p_val_sm = stests.ztest(sample_data,           # наші дані
                                    value=population_mean,  # μ₀ = 100
                                    alternative='larger')   # H₁: μ > 100

print(f"Результати z-тесту:")
print(f"  Z-статистика: {z_stat_sm:.3f}")
print(f"  P-value: {p_val_sm:.6f}")
print()

# Інтерпретація результатів
print("ІНТЕРПРЕТАЦІЯ:")
print("-" * 40)
if p_val_sm < alpha:
    print(f"✅ ВІДХИЛЯЄМО H₀")
    print(f"   p-value ({p_val_sm:.6f}) < α ({alpha})")
    print("   Висновок: Середній чек за липень 2025 року збільшився")
else:
    print(f"❌ НЕ ВІДХИЛЯЄМО H₀")
    print(f"   p-value ({p_val_sm:.6f}) ≥ α ({alpha})")
    print("   Висновок: Середній чек за липень 2025 не підвищився статистично значущо")

# Порівняння з ручними розрахунками
print("\n" + "=" * 60)
print("ПОРІВНЯННЯ З РУЧНИМИ РОЗРАХУНКАМИ:")
print("=" * 60)
print(f"Z-статистика:")
print(f"  Ручний розрахунок: {z_statistic:.3f}")
print(f"  Statsmodels:       {z_stat_sm:.3f}")
print(f"  Різниця:           {abs(z_statistic - z_stat_sm):.3f}")
print()
print("Примітка: Невелика різниця пояснюється тим, що ми")
print("використовуємо згенеровані дані, а не точне середнє 100")


ВИКОНАННЯ Z-ТЕСТУ:
Результати z-тесту:
  Z-статистика: -0.264
  P-value: 0.604288

ІНТЕРПРЕТАЦІЯ:
----------------------------------------
❌ НЕ ВІДХИЛЯЄМО H₀
   p-value (0.604288) ≥ α (0.05)
   Висновок: Середній чек за липень 2025 не підвищився статистично значущо

ПОРІВНЯННЯ З РУЧНИМИ РОЗРАХУНКАМИ:
Z-статистика:
  Ручний розрахунок: -0.241
  Statsmodels:       -0.264
  Різниця:           0.023

Примітка: Невелика різниця пояснюється тим, що ми
використовуємо згенеровані дані, а не точне середнє 100
